Import USA data

In [46]:
import pandas as pd
import glob
import os

path = os.getcwd() + "/Data/USA/covid-tracking-data-master/data/states_daily_4pm_et.csv"

#Data of healed are not present
#Home isolation are positive - hospitalized
#Intensive care and recovered is not present

columns = ["total_cases","daily_cases","actual_cases","total_dead","daily_dead",
                  "total_home_isolation","daily_home_isolation",
                  "total_hospitalized","daily_hospitalized","total_test","daily_test"]


dataset = pd.read_csv(path, index_col = 0)
dataset = dataset.fillna(0)

Grouping by date

In [47]:
dataset = dataset.groupby(["date"]).sum()
dataset

,positive,negative,pending,hospitalized,death,total
date,,,,,,
20200304,118.0,748.0,103.0,0.0,0.0,969
20200305,176.0,953.0,197.0,0.0,0.0,1326
20200306,223.0,1571.0,458.0,0.0,0.0,2252
20200307,341.0,1809.0,602.0,0.0,0.0,2752
20200308,417.0,2335.0,347.0,0.0,0.0,3099
20200309,584.0,3367.0,313.0,0.0,0.0,4264
20200310,778.0,3807.0,469.0,0.0,0.0,5054
20200311,1053.0,5978.0,563.0,0.0,27.0,7617
20200312,1315.0,7949.0,673.0,0.0,36.0,9966


Creating a new dataframe with our specific format

In [48]:
usa_dataset = pd.DataFrame().reindex_like(dataset)
usa_dataset = pd.DataFrame(index = dataset.index , columns = columns) 
usa_dataset = usa_dataset.fillna(0)
usa_dataset

,total_cases,daily_cases,actual_cases,total_dead,daily_dead,total_home_isolation,daily_home_isolation,total_hospitalized,daily_hospitalized,total_test,daily_test
date,,,,,,,,,,,
20200304,0,0,0,0,0,0,0,0,0,0,0
20200305,0,0,0,0,0,0,0,0,0,0,0
20200306,0,0,0,0,0,0,0,0,0,0,0
20200307,0,0,0,0,0,0,0,0,0,0,0
20200308,0,0,0,0,0,0,0,0,0,0,0
20200309,0,0,0,0,0,0,0,0,0,0,0
20200310,0,0,0,0,0,0,0,0,0,0,0
20200311,0,0,0,0,0,0,0,0,0,0,0
20200312,0,0,0,0,0,0,0,0,0,0,0


Filling the new dataframe with the avaiable data

In [116]:
day = []
for days in dataset.index:
    for days_usa in usa_dataset.index:
        try:
            #from 2nd row
            usa_dataset.loc[days]["daily_cases"] = dataset.loc[days]["positive"]-dataset.loc[days-1]["positive"]
            usa_dataset.loc[days]["daily_dead"] = dataset.loc[days]["death"]-dataset.loc[days-1]["death"]
            usa_dataset.loc[days]["daily_home_isolation"] = (dataset.loc[days]["positive"] - dataset.loc[days]["hospitalized"] - dataset.loc[days]["death"]) - (dataset.loc[days-1]["positive"] - dataset.loc[days-1]["hospitalized"] - dataset.loc[days-1]["death"]) #Instead of positive should be actual cases
            usa_dataset.loc[days]["daily_hospitalized"] = dataset.loc[days]["hospitalized"] - dataset.loc[days-1]["hospitalized"]
            usa_dataset.loc[days]["daily_test"] = dataset.loc[days]["total"] - dataset.loc[days-1]["total"]
        except:
            #First row
            usa_dataset.loc[days]["daily_cases"] = dataset.loc[days]["positive"]
            usa_dataset.loc[days]["daily_dead"] = dataset.loc[days]["death"]
            usa_dataset.loc[days]["daily_home_isolation"] = (dataset.loc[days]["positive"] - dataset.loc[days]["hospitalized"] - dataset.loc[days]["death"]) #Instead of positive should be actual cases
            usa_dataset.loc[days]["daily_hospitalized"] = dataset.loc[days]["hospitalized"]
            usa_dataset.loc[days]["daily_test"] = dataset.loc[days]["total"]
        finally:
            #Single row
            usa_dataset.loc[days]["total_cases"] = dataset.loc[days]["positive"]
            # missing recovered --> usa_dataset.loc[days]["actual_cases"] =
            usa_dataset.loc[days]["total_dead"] = dataset.loc[days]["death"]
            usa_dataset.loc[days]["total_home_isolation"] = dataset.loc[days]["positive"]-dataset.loc[days]["hospitalized"] - dataset.loc[days]["death"] #Instead of positive should be actual cases
            usa_dataset.loc[days]["total_hospitalized"] = dataset.loc[days]["hospitalized"]
            usa_dataset.loc[days]["total_test"] = dataset.loc[days]["total"]


usa_dataset

,total_cases,daily_cases,actual_cases,total_dead,daily_dead,total_home_isolation,daily_home_isolation,total_hospitalized,daily_hospitalized,total_test,daily_test
date,,,,,,,,,,,
20200304,118,118,0,0,0,118,118,0,0,969,969
20200305,176,58,0,0,0,176,58,0,0,1326,357
20200306,223,47,0,0,0,223,47,0,0,2252,926
20200307,341,118,0,0,0,341,118,0,0,2752,500
20200308,417,76,0,0,0,417,76,0,0,3099,347
20200309,584,167,0,0,0,584,167,0,0,4264,1165
20200310,778,194,0,0,0,778,194,0,0,5054,790
20200311,1053,275,0,27,27,1026,248,0,0,7617,2563
20200312,1315,262,0,36,9,1279,253,0,0,9966,2349
